In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import psycopg2
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
from shapely.geometry import MultiPolygon
from zipfile import ZipFile
import requests 
import sys
%matplotlib inline

In [2]:
import yaml

with open('../../config/postgres.yaml') as f:
    engine_configs = yaml.load(f, Loader=yaml.FullLoader)
    
try:
    engine = create_engine('postgresql://{username}:{password}@{host}:{port}/{dbname}'.format(**engine_configs))
except Exception as e:
    print("Uh oh, can't connect. Invalid dbname, user or password?")
    print(e)

In [3]:
def process_geometry_SQL_insert(gdf):
    gdf['geom'] = gdf['geometry'].apply(lambda x: WKTElement((MultiPolygon([x]) if x.geom_type == 'Polygon' else x).wkt, srid=4326))
    gdf = gdf.drop('geometry', 1)
    return gdf

# Often when reading in a ShapeFile from Basemap, you'll get: "ValueError: readshapefile can only handle 2D shape types"
# A trick can be to convert your geometry in your GeoPandas Dataframe and restoring the new flattened 2D geometry
# series back into a shapefile and try again.

# edit from http://stackoverflow.com/questions/33417764/basemap-readshapefile-valueerror  

from shapely.geometry import Polygon, MultiPolygon, shape, Point
def convert_3D_2D(geometry):
    '''
    Takes a GeoSeries of 3D Multi/Polygons (has_z) and returns a list of 2D Multi/Polygons
    '''
    new_geo = []
    for p in geometry:
        if p.has_z:
            if p.geom_type == 'Polygon':
                lines = [xy[:2] for xy in list(p.exterior.coords)]
                new_p = Polygon(lines)
                new_geo.append(new_p)
            elif p.geom_type == 'MultiPolygon':
                new_multi_p = []
                for ap in p:
                    lines = [xy[:2] for xy in list(ap.exterior.coords)]
                    new_p = Polygon(lines)
                    new_multi_p.append(new_p)
                new_geo.append(MultiPolygon(new_multi_p))
    return new_geo


In [4]:
CITY = 'boston'
NEIGHBORHOOD_SIZE = 805 # 805 OR 1609

In [11]:
bounds_gdf = gpd.read_file('../../data/boston/boundary/boundary.geojson')
bounds_gdf = bounds_gdf[['geometry']]
bounds_gdf['city'] = CITY
bounds_gdf.head()

,geometry,city
0,"MULTIPOLYGON (((-71.12655 42.37162, -71.12655 ...",boston1m


In [12]:
insert_gdf = process_geometry_SQL_insert(bounds_gdf)
insert_gdf.to_sql('boundary', engine, if_exists='append', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

## Spatial groups and blocks_group

In [13]:
block_groups_gdf = gpd.read_file('zip://../../data/boston/blocks_group/cb_2014_25_bg_500k.zip')
block_groups_gdf = block_groups_gdf[['GEOID', 'geometry']]
block_groups_gdf = block_groups_gdf.to_crs({'init': 'epsg:4326'}) 
block_groups_gdf.geometry = convert_3D_2D(block_groups_gdf.geometry)
block_groups_gdf.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,GEOID,geometry
0,250235601001,"POLYGON ((-70.79094 41.66259, -70.78026 41.666..."
1,250056321004,"POLYGON ((-71.33972 41.89009, -71.33236 41.888..."
2,250277162002,"POLYGON ((-71.69587 42.42999, -71.69358 42.431..."
3,250173103005,"POLYGON ((-71.32219 42.66181, -71.31913 42.662..."
4,250250203012,"POLYGON ((-71.06877 42.36579, -71.06514 42.364..."


In [14]:
# Exclude a block group that messes everything up
block_groups_gdf = block_groups_gdf[~block_groups_gdf.GEOID.isin(['250259818001', '250259815011'])]

In [15]:
block_groups_gdf = gpd.sjoin(block_groups_gdf, bounds_gdf, how="inner", op='intersects').drop('index_right', axis=1)

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [16]:
block_groups_gdf = block_groups_gdf.rename(columns={'GEOID': 'original_id'})
block_groups_gdf['city'] = CITY

In [17]:
insert_gdf = process_geometry_SQL_insert(block_groups_gdf)
insert_gdf.to_sql('temptable_{}'.format(CITY.lower()), engine, if_exists='replace', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

In [18]:
water_gdf = gpd.read_file('../../data/boston/blocks_group/hydrography.geojson')
water_gdf = water_gdf[['geometry']]
water_gdf = water_gdf.to_crs({'init': 'epsg:4326'}) 
water_gdf.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,geometry
0,"MULTIPOLYGON (((-70.99575 42.36290, -70.99572 ..."
1,"MULTIPOLYGON (((-71.01585 42.38034, -71.01518 ..."
2,"MULTIPOLYGON (((-71.03858 42.36488, -71.03858 ..."
3,"MULTIPOLYGON (((-71.11571 42.32665, -71.11572 ..."
4,"MULTIPOLYGON (((-71.11260 42.33313, -71.11259 ..."


In [19]:
insert_gdf = process_geometry_SQL_insert(water_gdf)
insert_gdf.to_sql('temptable2_{}'.format(CITY.lower()), engine, if_exists='replace', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

In [20]:
sql = """
UPDATE temptable_{tempname} AS b SET geom=ST_Multi(ST_Difference(b.geom, s.geom))
FROM (Select ST_Union(geom) as geom FROM temptable2_{tempname}) AS s
WHERE ST_Intersects(b.geom, s.geom) AND (NOT ST_Contains(s.geom, b.geom));
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [21]:
sql = """
INSERT INTO blocks_group (original_id, city, geom) 
SELECT s.original_id, s.city, ST_Multi(ST_Intersection(s.geom, b.geom))
FROM temptable_{tempname} as s
INNER JOIN boundary b ON ST_Intersects(s.geom, b.geom) AND (NOT ST_Touches(s.geom, b.geom)) AND b.city = s.city
where s.city='{city}' and ST_Area(ST_Intersection(s.geom, b.geom))/ST_Area(s.geom) >= 0.5;
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

### Neighborhoods

In [22]:
sql = """INSERT INTO spatial_groups (city, core_geom, core_id, lower_ids, spatial_name, approx_geom)
SELECT  a.city, a.geom as core_geom, a.bid as core_id, array_agg(b.bid), 'ego', ST_multi(ST_Union(b.geom))
FROM blocks_group a
INNER JOIN blocks_group b ON a.city = b.city AND (a.bid = b.bid OR ST_DWithin(a.geom::geography, ST_Centroid(b.geom)::geography, {distance}) OR st_touches(a.geom, b.geom)) 
where a.city='{city}'
GROUP BY a.bid, a.geom, a.city;
delete from spatial_groups where ST_Area(approx_geom::geography) < 250000 and spatial_name='ego';
""".format(city=CITY, tempname=CITY.lower(), distance=NEIGHBORHOOD_SIZE)

result = engine.execute(text(sql))

## Blocks

In [23]:
block_gdf = gpd.read_file('zip://../../data/boston/block/tl_2014_25_tabblock10.zip')
block_gdf = block_gdf[~block_gdf['GEOID10'].isin(['250259818001014', '250259812011003', '250259812011012'])]

block_gdf = block_gdf[['geometry']]
block_gdf = block_gdf.to_crs({'init': 'epsg:4326'}) 
block_gdf.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,geometry
0,"POLYGON ((-72.74106 42.60368, -72.74064 42.603..."
1,"POLYGON ((-70.95506 41.62354, -70.95450 41.623..."
2,"POLYGON ((-70.98886 42.26506, -70.98855 42.266..."
3,"POLYGON ((-70.80901 42.24785, -70.80881 42.248..."
4,"POLYGON ((-70.12356 41.26945, -70.12353 41.269..."


In [24]:
insert_gdf = process_geometry_SQL_insert(block_gdf)
insert_gdf.to_sql('temptable_{}'.format(CITY.lower()), engine, if_exists='replace', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

In [25]:
sql = """
insert into block (sp_id, geom, city, geog, greater_1sm) select bid, geom, city, geom::geography, ST_AREA(geom::geography)>2.59e+6 
from(
    SELECT bid, st_multi(geom) as geom, city, ROW_NUMBER() OVER (PARTITION BY geom ORDER by area DESC) AS r
    from (
        select b.bid, c.geom, b.city, ST_Area(ST_Intersection(b.geom, c.geom)) as area
        from temptable_{tempname} as c
        inner join blocks_group as b on ST_Intersects(b.geom, c.geom)
        where b.city = '{city}' AND ST_Area(ST_Intersection(b.geom, c.geom))/ST_Area(c.geom) > 0.5
    ) as dtable
) x
WHERE x.r = 1;
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [26]:
sql = """
UPDATE block AS b SET geom=ST_Multi(ST_Intersection(b.geom, s.geom))
FROM boundary AS s
WHERE ST_Intersects(b.geom, s.geom) AND b.city=s.city AND s.city='{city}' AND NOT ST_Contains(s.geom, b.geom);
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [27]:
## Some blocks_group do not have blocks
sql = """
DELETE FROM blocks_group bg
WHERE NOT EXISTS(SELECT * FROM block b WHERE b.sp_id = bg.bid) AND bg.city='{city}';
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Census

In [28]:
zip_file = ZipFile('../../data/boston/employment/ACS_14_5YR_B23025.zip')
zip_file.infolist()

[<ZipInfo filename='ACS_14_5YR_B23025.csv' compress_type=deflate file_size=100156 compress_size=20299>,
 <ZipInfo filename='ACS_14_5YR_B23025_metadata.csv' compress_type=deflate file_size=830 compress_size=218>,
 <ZipInfo filename='ACS_14_5YR_B23025.txt' compress_type=deflate file_size=3722 compress_size=1361>,
 <ZipInfo filename='aff_download_readme.txt' compress_type=deflate file_size=1951 compress_size=802>]

In [29]:
employment_df = pd.read_csv(zip_file.open('ACS_14_5YR_B23025.csv'))
employment_df = employment_df.rename(columns={'GEO.id2': 'original_id', 'HD01_VD03': 'inforce', 'HD01_VD04': 'employed', 'HD01_VD06': 'armed'})
employment_df = employment_df[['original_id', 'inforce', 'employed', 'armed']]
# Skip first header line
employment_df = employment_df[employment_df['original_id'] != 'Id2']

employment_df['inforce'] = employment_df['inforce'].astype(int)
employment_df['employed'] = employment_df['employed'].astype(int)
employment_df['armed'] = employment_df['armed'].astype(int)
employment_df.head()

,original_id,inforce,employed,armed
1,250250001001,738,709,0
2,250250001002,636,568,0
3,250250001003,285,248,0
4,250250001004,346,264,0
5,250250002011,497,479,0


In [30]:
zip_file = ZipFile('../../data/boston/population/ACS_14_5YR_B01003.zip')
zip_file.infolist()

[<ZipInfo filename='ACS_14_5YR_B01003.csv' compress_type=deflate file_size=71924 compress_size=9286>,
 <ZipInfo filename='ACS_14_5YR_B01003_metadata.csv' compress_type=deflate file_size=114 compress_size=91>,
 <ZipInfo filename='ACS_14_5YR_B01003.txt' compress_type=deflate file_size=3672 compress_size=1319>,
 <ZipInfo filename='aff_download_readme.txt' compress_type=deflate file_size=1951 compress_size=802>]

In [31]:
pop_df = pd.read_csv(zip_file.open('ACS_14_5YR_B01003.csv'))
pop_df = pop_df.rename(columns={'GEO.id2': 'original_id', 'HD01_VD01': 'population'})
pop_df = pop_df[['original_id', 'population']]
# Skip first header line
pop_df = pop_df[pop_df['original_id'] != 'Id2']

pop_df['population'] = pop_df['population'].astype(int)
pop_df.head()

,original_id,population
1,250250001001,1192
2,250250001002,875
3,250250001003,530
4,250250001004,667
5,250250002011,734


In [32]:
zip_file = ZipFile('../../data/boston/population/ACS_14_5YR_B25001.zip')
zip_file.infolist()

[<ZipInfo filename='ACS_14_5YR_B25001_with_ann.csv' compress_type=deflate file_size=71133 compress_size=8691>,
 <ZipInfo filename='ACS_14_5YR_B25001_metadata.csv' compress_type=deflate file_size=114 compress_size=91>,
 <ZipInfo filename='ACS_14_5YR_B25001.txt' compress_type=deflate file_size=3666 compress_size=1316>,
 <ZipInfo filename='aff_download_readme_ann.txt' compress_type=deflate file_size=1062 compress_size=463>]

In [33]:
dwellings_df = pd.read_csv(zip_file.open('ACS_14_5YR_B25001_with_ann.csv'), dtype={'GEO.id2': str})
dwellings_df = dwellings_df.rename(columns={'GEO.id2': 'original_id', 'HD01_VD01': 'dwellings'})
dwellings_df = dwellings_df[['original_id', 'dwellings']]

dwellings_df['dwellings'] = dwellings_df['dwellings'].astype(int)
dwellings_df.head()

,original_id,dwellings
0,250250001001,534
1,250250001002,435
2,250250001003,237
3,250250001004,413
4,250250002011,317


In [34]:
print(len(pop_df))
census_df = pd.merge(employment_df, pop_df, on='original_id')
census_df = pd.merge(census_df, dwellings_df, on='original_id')
print(len(census_df))
census_df.head()

646
646


,original_id,inforce,employed,armed,population,dwellings
0,250250001001,738,709,0,1192,534
1,250250001002,636,568,0,875,435
2,250250001003,285,248,0,530,237
3,250250001004,346,264,0,667,413
4,250250002011,497,479,0,734,317


In [35]:
census_df.to_sql('temptable_{tempname}'.format(tempname=CITY.lower()), engine, if_exists='replace', index=False)

In [36]:
sql = """
insert into census (bid, population, employed, inforce, tot_survey, dwellings, city) 
select b.bid, c.population, c.employed+c.armed, c.inforce, c.population, c.dwellings, '{city}' 
from temptable_{tempname} c 
inner join blocks_group b on b.original_id = c.original_id
where b.city='{city}';
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

### Residential stability

In [37]:
zip_file = ZipFile('../../data/boston/residential_stability/ACS_14_5YR_B07201.zip')
zip_file.infolist()

[<ZipInfo filename='ACS_14_5YR_B07201.csv' compress_type=deflate file_size=125261 compress_size=24559>,
 <ZipInfo filename='ACS_14_5YR_B07201_metadata.csv' compress_type=deflate file_size=3381 compress_size=400>,
 <ZipInfo filename='ACS_14_5YR_B07201.txt' compress_type=deflate file_size=4052 compress_size=1513>,
 <ZipInfo filename='aff_download_readme.txt' compress_type=deflate file_size=1951 compress_size=802>]

In [38]:
stab_df = pd.read_csv(zip_file.open('ACS_14_5YR_B07201.csv'))
stab_df = stab_df.rename(columns={'GEO.id2': 'original_id', 'HD01_VD01': 'total', 'HD01_VD02': 'stable'})
stab_df = stab_df[['original_id', 'total', 'stable']]
# Skip first header line
stab_df = stab_df[stab_df['original_id'] != 'Id2']

stab_df['total'] = stab_df['total'].astype(int)
stab_df['stable'] = stab_df['stable'].astype(int)
stab_df.head()

,original_id,total,stable
1,250250001001,1192,986
2,250250001002,875,729
3,250250001003,530,481
4,250250001004,667,610
5,250250002011,734,582


In [39]:
zip_file = ZipFile('../../data/boston/tenure/ACS_14_5YR_B25003.zip')
zip_file.infolist()

[<ZipInfo filename='ACS_14_5YR_B25003.csv' compress_type=deflate file_size=80277 compress_size=13367>,
 <ZipInfo filename='ACS_14_5YR_B25003_metadata.csv' compress_type=deflate file_size=312 compress_size=140>,
 <ZipInfo filename='ACS_14_5YR_B25003.txt' compress_type=deflate file_size=3668 compress_size=1312>,
 <ZipInfo filename='aff_download_readme.txt' compress_type=deflate file_size=1951 compress_size=802>]

In [40]:
tenure_df = pd.read_csv(zip_file.open('ACS_14_5YR_B25003.csv'))
tenure_df = tenure_df.rename(columns={'GEO.id2': 'original_id', 'HD01_VD01': 'total2', 'HD01_VD02': 'owner'})
tenure_df = tenure_df[['original_id', 'total2', 'owner']]
# Skip first header line
tenure_df = tenure_df[tenure_df['original_id'] != 'Id2']

tenure_df['total2'] = tenure_df['total2'].astype(int)
tenure_df['owner'] = tenure_df['owner'].astype(int)
tenure_df.head()

,original_id,total2,owner
1,250250001001,492,154
2,250250001002,364,100
3,250250001003,213,43
4,250250001004,299,54
5,250250002011,284,142


In [41]:
res_stability_df = pd.merge(stab_df, tenure_df, on='original_id')
res_stability_df.head()

,original_id,total,stable,total2,owner
0,250250001001,1192,986,492,154
1,250250001002,875,729,364,100
2,250250001003,530,481,213,43
3,250250001004,667,610,299,54
4,250250002011,734,582,284,142


In [42]:
res_stability_df.to_sql('temptable_{tempname}'.format(tempname=CITY.lower()), engine, if_exists='replace', index=False)

In [43]:
sql = """
INSERT INTO residential_stability (bid, city, total, stable, total2, owner) 
SELECT b.bid, '{city}', c.total, c.stable, c.total2, c.owner 
FROM temptable_{tempname} c 
INNER JOIN blocks_group b ON b.original_id = c.original_id
WHERE b.city = '{city}';
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

### Ethnic diversity

In [44]:
zip_file = ZipFile('../../data/boston/ethnic_diversity/ACS_14_5YR_B02001.zip')
zip_file.infolist()

[<ZipInfo filename='ACS_14_5YR_B02001.csv' compress_type=deflate file_size=108221 compress_size=21897>,
 <ZipInfo filename='ACS_14_5YR_B02001_metadata.csv' compress_type=deflate file_size=1373 compress_size=336>,
 <ZipInfo filename='ACS_14_5YR_B02001.txt' compress_type=deflate file_size=3660 compress_size=1307>,
 <ZipInfo filename='aff_download_readme.txt' compress_type=deflate file_size=1951 compress_size=802>]

In [45]:
eth_df = pd.read_csv(zip_file.open('ACS_14_5YR_B02001.csv'))
eth_df = eth_df.rename(columns={'GEO.id2': 'original_id', 'HD01_VD02': 'white', 'HD01_VD03': 'black', 'HD01_VD04': 'native', 'HD01_VD05': 'asian', 'HD01_VD06': 'native2', 
                               'HD01_VD08': 'o1', 'HD01_VD09': 'o2', 'HD01_VD10': 'o3'})
eth_df = eth_df[['original_id', 'white', 'black', 'asian', 'native', 'native2', 'o1', 'o2', 'o3']]
# Skip first header line
eth_df = eth_df[eth_df['original_id'] != 'Id2']

for x in ['white', 'black', 'asian', 'native', 'native2', 'o1', 'o2', 'o3']:
    eth_df[x] = eth_df[x].astype(int)
    
eth_df['other'] = eth_df['o1'] + eth_df['o2'] + eth_df['o3']
eth_df = eth_df.drop(['o1', 'o2', 'o3'], axis=1)
eth_df.head()

,original_id,white,black,asian,native,native2,other
1,250250001001,887,158,129,0,0,36
2,250250001002,609,8,157,0,0,0
3,250250001003,451,6,66,0,0,14
4,250250001004,488,0,74,0,0,30
5,250250002011,559,47,69,0,0,36


In [46]:
eth_df.to_sql('temptable_{tempname}'.format(tempname=CITY.lower()), engine, if_exists='replace', index=False)

In [47]:
sql = """
INSERT INTO ethnic_diversity (bid, city, race1, race2, race3, race4, race5, race6) 
SELECT b.bid, '{city}', c.white, c.black, c.native, c.asian, c.native2, c.other
FROM temptable_{tempname} c 
INNER JOIN blocks_group b ON b.original_id = c.original_id
where b.city='{city}';
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

### Poverty

In [48]:
zip_file = ZipFile('../../data/boston/poverty/ACS_14_5YR_C17002.zip')
zip_file.infolist()

[<ZipInfo filename='ACS_14_5YR_C17002.csv' compress_type=deflate file_size=101494 compress_size=21834>,
 <ZipInfo filename='ACS_14_5YR_C17002_metadata.csv' compress_type=deflate file_size=759 compress_size=222>,
 <ZipInfo filename='ACS_14_5YR_C17002.txt' compress_type=deflate file_size=3742 compress_size=1373>,
 <ZipInfo filename='aff_download_readme.txt' compress_type=deflate file_size=1951 compress_size=802>]

In [49]:
pov_df = pd.read_csv(zip_file.open('ACS_14_5YR_C17002.csv'))
pov_df = pov_df.rename(columns={'GEO.id2': 'original_id', 'HD01_VD01': 'total', 'HD01_VD02': 'p50', 'HD01_VD03': 'p99'})
pov_df = pov_df[['original_id', 'total', 'p50', 'p99']]
# Skip first header line
pov_df = pov_df[pov_df['original_id'] != 'Id2']

for x in ['total', 'p50', 'p99']:
    pov_df[x] = pov_df[x].astype(int)
    
pov_df['poors'] = pov_df['p50'] + pov_df['p99']
pov_df = pov_df.drop(['p50', 'p99'], axis=1)
pov_df.head()

,original_id,total,poors
1,250250001001,1183,248
2,250250001002,875,124
3,250250001003,530,27
4,250250001004,667,156
5,250250002011,734,62


In [50]:
pov_df.to_sql('temptable_{tempname}'.format(tempname=CITY.lower()), engine, if_exists='replace', index=False)

In [51]:
sql = """
INSERT INTO poverty_index (bid, city, total, poors) 
SELECT b.bid, '{city}', c.total, c.poors
FROM temptable_{tempname} c 
INNER JOIN blocks_group b ON b.original_id = c.original_id
where b.city='{city}';
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Buildings

In [52]:
bld_gdf = gpd.read_file('zip://../../data/boston/buildings/Buildings.zip')
print(len(bld_gdf))
bld_gdf = bld_gdf[bld_gdf["IEL_TYPE"] == 'BLDG']
print(len(bld_gdf))
bld_gdf = bld_gdf.rename(columns={'PART_HEIGH': 'height', 'PART_FLOOR': 'floors'})

bld_gdf = bld_gdf[['height', 'geometry','floors']]
bld_gdf = bld_gdf.to_crs({'init': 'epsg:4326'}) 

bld_gdf['floors'] = bld_gdf['floors'].astype('float32')

bld_gdf.head()

129370
102179


/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,height,geometry,floors
0,21.799999,"POLYGON ((-71.16550 42.34636, -71.16549 42.346...",NaN
1,34.299999,"POLYGON ((-71.15360 42.34629, -71.15360 42.346...",NaN
2,25.600000,"POLYGON ((-71.15490 42.34607, -71.15489 42.346...",NaN
3,24.400000,"POLYGON ((-71.15367 42.34617, -71.15366 42.346...",NaN
4,27.799999,"POLYGON ((-71.15387 42.34527, -71.15387 42.345...",NaN


In [53]:
insert_gdf = process_geometry_SQL_insert(bld_gdf)
insert_gdf.to_sql('temptable_{tempname}'.format(tempname=CITY.lower()), engine, if_exists='replace', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

In [54]:
sql = """
UPDATE temptable_{tempname} p SET geom=ST_Multi(ST_buffer(p.geom, 0.0)) 
WHERE NOT st_isvalid(p.geom)
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [55]:
sql = """
INSERT INTO building (bid, city, geom, height, area) 
SELECT bid, '{city}', geom, height, barea
FROM (
    SELECT bid, geom, height, barea, ROW_NUMBER() OVER (PARTITION BY geom ORDER BY area DESC) AS r
    from (
        SELECT p.geom, p.height, ST_Area(p.geom::geography) as barea, d.bid, ST_Area(ST_Intersection(p.geom, d.geom)) as area
        FROM temptable_{tempname} as p
        INNER JOIN blocks_group as d on ST_Intersects(p.geom, d.geom) AND NOT ST_Touches(p.geom, d.geom)
        WHERE d.city = '{city}' 
        ) as dtable
    order by area
) x
WHERE x.r = 1;
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Land use

In [56]:
land_gdf = gpd.read_file('zip://../../data/boston/land_use/Parcels_2016_Data_Full.zip')

land_gdf['sqftmain'] = land_gdf['LIVING_ARE'].fillna(0) + land_gdf['GROSS_AREA'].fillna(0)

# Remove streets
land_gdf = land_gdf[~((land_gdf["OWNER"]=='COMMONWEALTH OF MASS') & (land_gdf["YR_BUILT"]=='0'))]

land_gdf = land_gdf.rename(columns={'PID_LONG': 'pid', 'LU': 'usecode', 'PTYPE': 'usecode2', 'YR_BUILT': 'yearbuilt', 'AV_TOTAL': 'value'})

land_gdf = land_gdf[['pid', 'sqftmain', 'usecode', 'usecode2', 'yearbuilt', 'geometry', 'value']]

land_gdf = land_gdf[~(land_gdf['geometry'].isnull())]
land_gdf = land_gdf.to_crs({'init': 'epsg:4326'}) 

land_gdf.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,pid,sqftmain,usecode,usecode2,yearbuilt,geometry,value
0,0100001000,5555.0,Residential Three Family,105,1900,"POLYGON ((-71.03250 42.37933, -71.03257 42.379...",365200
1,0100002000,5606.0,Residential Three Family,105,1920,"POLYGON ((-71.03244 42.37935, -71.03250 42.379...",402200
2,0100003000,5660.0,Residential Three Family,105,1905,"POLYGON ((-71.03237 42.37938, -71.03244 42.379...",366100
3,0100004000,5136.0,Residential Three Family,105,1900,"POLYGON ((-71.03231 42.37940, -71.03237 42.379...",349700
4,0100005000,6246.0,Residential Two Family,104,1900,"POLYGON ((-71.03240 42.37955, -71.03234 42.379...",395500


In [57]:
land_gdf_condos = land_gdf[land_gdf['usecode'] == 'Residential Condo Unit'].copy()
land_gdf_condos_geom = land_gdf_condos[['pid', 'geometry']].drop_duplicates(subset='pid')
land_gdf_condos_nogeom = land_gdf_condos.groupby(['pid', 'usecode', 'usecode2', 'yearbuilt'], as_index=False).mean()
land_gdf_condos = pd.merge(land_gdf_condos_geom, land_gdf_condos_nogeom, on='pid')[['pid', 'sqftmain', 'usecode', 'usecode2', 'yearbuilt', 'geometry', 'value']]

land_gdf = pd.concat((land_gdf[land_gdf['usecode'] != 'Residential Condo Unit'], land_gdf_condos))
land_gdf.head()

,pid,sqftmain,usecode,usecode2,yearbuilt,geometry,value
0,0100001000,5555.0,Residential Three Family,105,1900,"POLYGON ((-71.03250 42.37933, -71.03257 42.379...",365200.0
1,0100002000,5606.0,Residential Three Family,105,1920,"POLYGON ((-71.03244 42.37935, -71.03250 42.379...",402200.0
2,0100003000,5660.0,Residential Three Family,105,1905,"POLYGON ((-71.03237 42.37938, -71.03244 42.379...",366100.0
3,0100004000,5136.0,Residential Three Family,105,1900,"POLYGON ((-71.03231 42.37940, -71.03237 42.379...",349700.0
4,0100005000,6246.0,Residential Two Family,104,1900,"POLYGON ((-71.03240 42.37955, -71.03234 42.379...",395500.0


In [58]:
wrong_pids = ['0303430000', '0305362000', '0305364000', '0305507000',
       '0305510000', '0305520000', '0306232000', '0400575000',
       '0400592000', '0401141000', '0401082000', '2100396010',
       '0401140000', '0500355000', '0500476000', '0500739001',
       '0500706010', '0501004000', '0501071000', '0501073000',
       '0600001003', '0600002000', '0600002001', '0600004003',
       '0600005000', '0602769010', '0702849000', '0703355000',
       '0703428000', '0703501003', '0703583000', '0703607000',
       '0703677000', '0801021230', '0801070000', '0902819000',
       '1100001000', '1100873000', '1100873010', '1100896030',
       '1101197000', '1101243000', '1102529000', '1102602000',
       '1102618000', '1102638000', '1301107000', '1301199000',
       '1303321001', '1400109100', '1400116100', '1401140100',
       '1401603000', '1402330100', '1402931000', '1700624100',
       '1701590200', '1701604000', '1702190100', '1800734100',
       '1800970000', '1801039000', '1801981000', '1803773000',
       '1804382000', '1805622000', '1806598000', '1806637000',
       '1808197000', '1809226000', '1809477000', '1809793150',
       '1810095000', '1810602100', '1811037000', '1811089000',
       '1811141100', '1811546000', '1812149000', '1812161001',
       '1812738000', '1812974500', '1813019000', '1902798000',
       '1902805001', '1902822000', '1905052010', '2005233000', '0700072010', '0700072030', '2001613500', '2002318500',
       '2002287000', '2005507000', '2005717000', '2005723000',
       '2008811000', '2008871001', '2008917500', '2009155500',
       '2009216300', '2009226000',  '2201868000', '2200105000',
       '2201906100', '2202727000', '2202770000', '2203417000',
       '0103679000', '0103680001', '0103918055', '0103920055',
       '0104165000', '0104166000', '0105346050', '0105346060',
       '0202645000', '0202672000', '0202699000', '0202732000',
       '0202830000', '0202866000', '0202926000', '0203591001',
       '0203591002', '0303048000', '0402123010', '0402133010',
       '0402170010', '0402235200', '0402235300', '0402236100',
       '0402249000', '0402342500', '0504175003', '0504175004',
       '0504175005', '0504198001', '1500574000', '1600113000',
       '1600257000', '1600258000', '1600603000', '1600776000',
       '1601395000', '1601559000', '1602552000', '1602561000',
       '1602563000', '1602723000', '1602775000', '1602784000',
       '1602784001', '1700822000', '1700841000', '1701061000',
       '1701152000', '0100433000', '0100435005', '0100435010', '0100436001',
       '0100485010', '0101392000', '0101392001', '0101393000',
       '0101562000', '0103705020', '0104290000', '0104295001',
       '0104322000', '0104413000', '0104419000', '0201800010',
       '0201916000', '0201930000', '0202140000', '0202140100',
       '0202146000', '0202147000', '0202148000', '0202150010',
       '0202178001', '0202178002', '0202178050', '0202179000',
       '0202180000', '0202190000', '0202755015', '0203962000',
       '0203962001', '0203968000', '0203963100', '0300944000',
       '0301913000', '0603413000', '0603419001', '0902888000',
       '0902909000', '1302219000', '1302220000', '1302432001',
       '1302543000', '1302543001', '1303390000', '1400359000',
       '1400368000', '1405199002', '1500012000', '1503090000',
       '1600002000', '1600003000', '1600004000', '1600007001',
       '1600008001', '1600008010', '1600010000', '1600220000',
       '1600231000', '1600232000', '1600251000', '1602416000',
       '1602417000', '1602455001', '1602562000', '1602569000',
       '1602615800', '1602737000', '1602774000', '1604855000',
       '1604945000', '1604974000', '1605385000', '1703702000',
       '1800513000', '1800968000', '1801034000', '0300448000', '0301130010', '0301133010', '0301136010',
       '0300520000', '0300521000', '0300521010', '0300540000',
       '0300541000', '0300550000', '0301059000', '0301068010',
       '0301072010', '0301074000', '0301121000', '0301121010',
       '0301687000', '0301922000', '0301927050', '0301935003',
       '0302600000', '0302604000', '0303307000', '0303324000',
       '0303435000', '0304981000', '0304991000', '0305361000',
       '0305361003', '0305366000', '0306869000', '0400815000',
       '0401041000', '0502232000', '0502239000', '0502252000',
       '0502495001', '0503043000', '0503704001', '0503704002',
       '0503704003', '0504145000', '0504155000', '0600006010',
       '0600207010', '0600223010', '0600232000', '0600233000',
       '0600240000', '0600248000', '0600257000', '0601245000',
       '0601252010', '0601253000', '0602769013', '0602769017',
       '0602769021', '0700013000', '0700072000', '0700072020',
       '0700072025', '0801024000', '0801024006', '0801024008',
       '0801069000', '1100750000', '1101233040', '1101238000',
       '1101240000', '1102927000', '1102928050', '1102943000',
       '1102944000', '1102945000', '1102946000', '1102947000',
       '1102948000', '1303400000', '1303448000', '1500022000',
       '1500563000', '1500569000', '1600593000', '1600578000',
       '1600748000', '1600756000', '1604320000', '1809440000',
       '1809792050', '1809793010', '1809793100', '1812161000',
       '1812170000', '1812369000', '1812369001', '1813018000',
       '1904603000', '1904604000', '1904606000', '2100063000',
       '2100069015', '2100069035', '2100069050', '2100069055',
       '2100069075', '2100171001', '2100272000', '2100357000',
       '2100357001', '0304995000', '0700073000', '0702812000', '1303259000',
       '1303320000', '1303322000', '0103712000', '0106217000', '0106291000', '0203615000',
       '0203964001', '0203980000', '0301523001', '0301532000',
       '0301570000', '0301927510', '0301910000', '0301911000',
       '0303792000', '0304351000', '0700028001', '0700239000',
       '0702740000', '0702740001', '0703337000', '0703354000',
       '0801404000', '0801415000', '1302240000', '1600225000',
       '1600252000', '1600258001', '1602376001', '1602418000',
       '1602412000', '1602492000', '1602568000', '1602590001',
       '1602735001', '1812603000', '1812736000', '2100396000',
       '2200103000', '0201877000', '0201877001', '0201878001', '0201928000',
       '0201929000', '0202179050', '0202191000', '0203962002',
       '0203962003', '0203962004', '0203962005', '0203962006',
       '0203962007', '0203962008', '0203962009', '0203962020',
       '0203963001', '0203963002', '0400576000', '0500705000',
       '1102642001']
land_gdf = land_gdf[~land_gdf.pid.isin(wrong_pids)]

In [59]:
land_gdf['landuse'] = 'commercial'

land_gdf.loc[(land_gdf['usecode'].str[:1].isin({'A', 'R'})) & (land_gdf['usecode'] != 'Agricultural/Horticultural') & (land_gdf['usecode'] != 'Residential Land'), 'landuse'] = 'residential'
land_gdf.loc[land_gdf['usecode'].isin({'Condominium Master', 'Condominium Parking'}), 'landuse'] = 'residential'
 
land_gdf.loc[(~land_gdf['usecode2'].isin({353, 905, 906})) & (land_gdf['usecode'].isin({'Commercial', 'Industrial', 'Commercial Condo Unit'})) & ((land_gdf['usecode2'] > 383) | (land_gdf['usecode2'] < 360)), 'landuse'] = 'commercial'
land_gdf.loc[(~land_gdf['usecode2'].isin({353, 905, 906})) & ((land_gdf['usecode2'] >= 950) & (land_gdf['usecode2'] <= 965)), 'landuse'] = 'commercial'
land_gdf.loc[(land_gdf['usecode2'] >= 972) & (land_gdf['usecode2'] <= 983), 'landuse'] = 'commercial'

land_gdf.loc[(land_gdf['usecode'] == 'Exempt') & (~((land_gdf['usecode2'] >= 950) & (land_gdf['usecode2'] <= 965))) & (~((land_gdf['usecode2'] >= 972) & (land_gdf['usecode2'] <= 983))), 'landuse'] = 'recreational'
land_gdf.loc[((land_gdf['usecode2'] >= 360) & (land_gdf['usecode2'] <= 383)), 'landuse'] = 'recreational'
land_gdf.loc[land_gdf['usecode2'].isin({353, 905, 906}), 'landuse'] = 'recreational'

land_gdf.loc[(land_gdf['usecode'].isin({'Residential Land', 'Commercial Land'})), 'landuse'] = 'vacant'

# Some fixes
land_gdf.loc[land_gdf['usecode2'].isin({908}), 'landuse'] = 'residential'
land_gdf.loc[land_gdf['usecode2'].isin({904, 902}), 'landuse'] = 'commercial'


mixed = land_gdf[land_gdf['usecode'].isin({'Mixed Use Res/Comm'})].copy()
mixed2 = mixed.copy()

mixed.loc[:, 'landuse'] = 'residential'
mixed.loc[:, 'sqftmain'] /= 2
mixed2.loc[:, 'landuse'] = 'commercial'
mixed2.loc[:, 'sqftmain'] /= 2

land_gdf = pd.concat((land_gdf[~(land_gdf['usecode'].isin({'Mixed Use Res/Comm'}))], mixed, mixed2))
land_gdf.head()

,pid,sqftmain,usecode,usecode2,yearbuilt,geometry,value,landuse
0,0100001000,5555.0,Residential Three Family,105,1900,"POLYGON ((-71.03250 42.37933, -71.03257 42.379...",365200.0,residential
1,0100002000,5606.0,Residential Three Family,105,1920,"POLYGON ((-71.03244 42.37935, -71.03250 42.379...",402200.0,residential
2,0100003000,5660.0,Residential Three Family,105,1905,"POLYGON ((-71.03237 42.37938, -71.03244 42.379...",366100.0,residential
3,0100004000,5136.0,Residential Three Family,105,1900,"POLYGON ((-71.03231 42.37940, -71.03237 42.379...",349700.0,residential
4,0100005000,6246.0,Residential Two Family,104,1900,"POLYGON ((-71.03240 42.37955, -71.03234 42.379...",395500.0,residential


In [60]:
# Some mistakes in the dataset
land_gdf.loc[land_gdf.pid == '1902795000', 'landuse'] = 'recreational'
land_gdf.loc[land_gdf.pid == '2005207000', 'landuse'] = 'recreational'

land_gdf.loc[land_gdf.pid == '0104126000', 'landuse'] = 'commercial'

land_gdf.loc[land_gdf.pid.isin(['0104126001',
'0603417000',
'0603411000',
'0603412000',
'0603407000',
'0104446010', '0104447010', '0303027000', '0203505600',
       '0203517700', '0203517800', '0302990000', '0303004000',
       '0303026000', '0303028300', '0303029200', '0303030500',
       '0303035000', '0303036000', '0303036001', '0303036005',
       '0303037000', '0303044000', '0303050000', '0602828001',
       '0602828002', '0602829000', '0305364000', '0602831000',
       '0305365000', '0602642005', '0602673100', '0602672000',
       '0602673001', '0602673002', '0602673002', '0602673002',
       '0602673002', '0602673120', '0602673150', '0602674000',
       '0602674008', '0602674010', '0602674013', '0602674014',
       '0602674015', '0602674019', '0602674020', '0602674025',
       '0602674030', '0602674040', '0602674050', '0602675000',
       '0602675001', '0602676001', '0602676005', '0602676010',
       '0602676025', '0602676035', '0602678000', '0602678020',
       '0602678500', '0602678700', '0602679000', '0602680100',
       '0602680120', '0602680200', '0602683000', '0602746010',
       '0602747010', '0602749100', '0602756020', '0602769030',
       '0602830000', '0602769018', '0602769024', '0602769100',
       '0602769150', '0602774010', '0602782010', '0602798010',
       '0602812100', '0602813000', '0602813003', '0602814000',
       '0602814001', '0602814002', '0602814003', '0602815000',
       '0602815001', '0602816000', '0602817000', '0602817005',
       '0602817020', '0602819001', '0602824000', '0602826000',
       '0602827000', '0602827001', '0602827002', '0602828000',
       '0602834002', '0602835000', '0602836010', '0602838050',
       '0602839005', '0602839010', '0201791000', '0201796000', '0201797001', '0202755004',
       '0202754000', '0202755015', '0202756000', '0202756001',
       '0203594100', '0301925000', '0302573000', '0302605000',
       '0302615000', '0305363100', '0401620000', '0401666000',
       '0600004010', '0600005001', '0600116000', '0600117000',
       '0600118000', '0600119000', '0600121000', '0600123000',
       '0600124000', '0600125000', '0600126000', '0600164000',
       '0600312000', '0801024040', '0801024107', '0801024035',
       '0801024036', '0801024037', '0801834000', '0801854010',
       '0801878010', '0801903010', '0801973000', '0801984010',
       '0802375000', '0802380000', '1400359000', '1400368000', '1405199002', '1800513000',
       '1800968000', '1801034000', '0100433000', '0100435005', '0100435010', '0100436001',
       '0100485010', '0101392000', '0101392001', '0101393000',
       '0101562000', '0103705020', '0104290000', '0104295001',
       '0104322000', '0104413000', '0104419000', '0201800010',
       '0201916000', '0201930000', '0202140000', '0202140100',
       '0202146000', '0202147000', '0202148000', '0202150010',
       '0202178001', '0202178002', '0202178050', '0202179000',
       '0202180000', '0202190000', '0202755015', '0203962000',
       '0203962001', '0203968000', '0203963100', '0500036000',
       '0300460000', '0300475000', '0300944000', '0301686000',
       '0301913000', '0302722000', '0302723000', '0302714000',
       '0305560010', '0305599010', '0305599030', '0306500000',
       '0306264000', '0306499000', '0401841000', '0401849000',
       '0401850000', '0401996000', '0402011020', '0500010000',
       '0500035000', '0603413000', '0603419001', '0801026000',
       '0801026001', '0801026003', '0801026004', '0801026100',
       '0801027000', '0801027010', '0801027011', '0801027012',
       '0801029100', '0801032005', '0801032010', '0801038002',
       '0801147000', '0801147010', '0801148000', '0801149000',
       '0801150000', '0801151000', '0801153010', '0801167000',
       '0801167020', '0801170010', '0801408000', '0902643000',
       '0902888000', '0902909000', '1001670000', '1001781000',
       '1002199000', '1302219000', '1302220000', '1302432001',
       '1302543000', '1302543001', '1303390000', '1400359000',
       '1400368000', '1405199002', '1500012000', '1503090000',
       '1600002000', '1600003000', '1600004000', '1600007001',
       '1600008001', '1600008010', '1600010000', '1600220000',
       '1600231000', '1600232000', '1600251000', '1602416000',
       '1602417000', '1602455001', '1602562000', '1602569000',
       '1602615800', '1602737000', '1602774000', '1604855000',
       '1604945000', '1604974000', '1605385000', '1703702000',
       '1800513000', '1800968000', '1801034000', '0100433000', '0100435005', '0100435010', '0100436001',
       '0100485010', '0101392000', '0101392001', '0101393000',
       '0101562000', '0103705020', '0104290000', '0104295001',
       '0104322000', '0104413000', '0104419000', '0201800010',
       '0201916000', '0201930000', '0202140000', '0202140100',
       '0202146000', '0202147000', '0202148000', '0202150010',
       '0202178001', '0202178002', '0202178050', '0202179000',
       '0202180000', '0202190000', '0202755015', '0203962000',
       '0203962001', '0203968000', '0203963100', '0500036000',
       '0300460000', '0300475000', '0300944000', '0301686000',
       '0301913000', '0302722000', '0302723000', '0302714000',
       '0305560010', '0305599010', '0305599030', '0306500000',
       '0306264000', '0306499000', '0401841000', '0401849000',
       '0401850000', '0401996000', '0402011020', '0402043300',
       '0402043000', '0402043100', '0402043200', '0402043400',
       '0402043500', '0402043600', '0402043700', '0500010000',
       '0500035000', '0603413000', '0603419001', '0801026000',
       '0801026001', '0801026003', '0801026004', '0801026100',
       '0801027000', '0801027010', '0801027011', '0801027012',
       '0801029100', '0801032005', '0801032010', '0801038002',
       '0801147000', '0801147010', '0801148000', '0801149000',
       '0801150000', '0801151000', '0801153010', '0801167000',
       '0801167020', '0801170010', '0801408000', '0902643000',
       '0902888000', '0902909000', '1001670000', '1001781000',
       '1002199000', '1302219000', '1302220000', '1302432001',
       '1302543000', '1302543001', '1303390000', '1400359000',
       '1400368000', '1405199002', '1500012000', '1503090000',
       '1600002000', '1600003000', '1600004000', '1600007001',
       '1600008001', '1600008010', '1600010000', '1600220000',
       '1600231000', '1600232000', '1600251000', '1602416000',
       '1602417000', '1602455001', '1602562000', '1602569000',
       '1602615800', '1602737000', '1602774000', '1604855000',
       '1604945000', '1604974000', '1605385000', '1703702000',
       '1800513000', '1800968000', '1801034000', '0700072000', '1100750000', '1101233040', '1101238000',
       '1101240000', '1102927000', '1102928050', '1102943000',
       '1102944000', '1102945000', '1102946000', '1102947000',
       '1102948000', '1303400000', '1303448000', '1904603000',
       '1904604000', '1904606000', '0301570000', '0302726500', '0302782000', '0302856000',
       '0302959000', '0303689000', '0303905000', '0304585010',
       '0304590010', '0304995000', '0305229001', '0305231000',
       '0305232000', '0305233000', '0305234000', '0305235000',
       '0305236000', '0305238000', '0305239000', '0305240000',
       '0305242000', '0305246000', '0305248001', '0305249000',
       '0305250000', '0305648000', '0601410000', '0700073000',
       '0702812000', '0802472000', '0802475000', '0903082200',
       '1101465010', '1201261000', '1303259000', '1303320000',
       '1303322000', '1703652000', '1703653000', '1703654000',
       '1703705000', '1800003000', '1800005000', '1801014000', '0202170000', '0203963050', '0203964000', '0302886000',
       '0304585010', '0304590010', '0304890000', '0305648000',
       '0500774000', '1812152010', '2009294000', '2202495000',
       '2202496000', '2202497000']), 'landuse'] = 'commercial'


land_gdf.loc[land_gdf.pid.isin(['0702845000', '0702846000', '0702847000', '0702853000',
       '1800041000', '1800044000', '1800112000', '1800112001',
       '1800113300', '1800113400', '1800113500', '1800257000',
       '1800283010', '1800283020', '1800283030']), 'landuse'] = 'residential'

land_gdf.loc[land_gdf.pid.isin(['1902186000', '1902823000', '1402863000', '1402883000',
                               '1001214000', '0401994001', '0305913000', '0306535000', '0401994001', '0500812000',
       '0501259000', '0502824500', '0502875500', '0502921500',
       '0502939500', '0503371500', '0503391500', '0503564500',
       '0503575500', '0503817500', '0503893500', '0801208000',
       '0801504500', '0801979000', '0801981000', '0802424000',
       '0802922000', '0902025000', '0902025010', '0902025020',
       '0902040000', '0902040010', '0902041000', '0902755000',
       '0903338000', '0903340000', '1000428000', '1000432000',
       '1000434000', '1000435000', '1000436000', '1000437000',
       '1000530000', '1000531000', '1000532000', '1001125000',
       '1001126000', '1001127000', '1001128000', '1001129000',
       '1001130000', '1001131000', '1001132000', '1001133000',
       '1001134000', '1001136000', '1001137000', '1001214000',
       '1001491100', '1202881000', '1402863000', '1402883000',
       '1808199001', '1808200000', '1811040000', '1811041000',
       '1811042000', '1811043000', '1811439000', '1811449000',
       '1811450000', '1900350000', '1900248000', '1900315500',
       '1902186000', '1902823000', '2003590500', '2005121100',
       '2005137000', '2005138000', '2005140000', '2005141000',
       '2005143000', '2005144000', '2005145000', '2005146000',
       '2005147000', '2005148000', '2005149000', '2005150000',
       '2005153000', '2005154000', '2005156000', '2005157000',
       '2005158000', '2005159001', '2005160000', '2005161000',
       '2005162000', '2005163000', '2005309000', '2008967000',
       '2009010000']), 'landuse'] = 'recreational'


In [61]:
land_gdf = land_gdf.reset_index()
land_gdf.head()

,index,pid,sqftmain,usecode,usecode2,yearbuilt,geometry,value,landuse
0,0,0100001000,5555.0,Residential Three Family,105,1900,"POLYGON ((-71.03250 42.37933, -71.03257 42.379...",365200.0,residential
1,1,0100002000,5606.0,Residential Three Family,105,1920,"POLYGON ((-71.03244 42.37935, -71.03250 42.379...",402200.0,residential
2,2,0100003000,5660.0,Residential Three Family,105,1905,"POLYGON ((-71.03237 42.37938, -71.03244 42.379...",366100.0,residential
3,3,0100004000,5136.0,Residential Three Family,105,1900,"POLYGON ((-71.03231 42.37940, -71.03237 42.379...",349700.0,residential
4,4,0100005000,6246.0,Residential Two Family,104,1900,"POLYGON ((-71.03240 42.37955, -71.03234 42.379...",395500.0,residential


In [62]:
ins_gdf = process_geometry_SQL_insert(land_gdf.copy())
ins_gdf.to_sql('temptable_{tempname}'.format(tempname=CITY.lower()), engine, if_exists='replace', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

In [63]:
sql = """
UPDATE temptable_{tempname} p SET geom=ST_Multi(ST_buffer(p.geom, 0.0)) 
WHERE NOT st_isvalid(p.geom);
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [64]:
sql = """
INSERT INTO land_uses (bid, city, use_type, area) 
SELECT bid, '{city}', landuse, SUM(area) 
FROM (
    SELECT bid, landuse, area, ROW_NUMBER() OVER (PARTITION BY index ORDER BY area DESC) AS r
    from (
        SELECT p.index, p.pid, p.landuse, p.sqftmain, d.bid, ST_Area(ST_Intersection(p.geom, d.geom)::geography) as area
        FROM temptable_{tempname} as p
        INNER JOIN blocks_group as d on ST_Intersects(p.geom, d.geom) AND NOT ST_Touches(p.geom, d.geom)
        WHERE d.city = '{city}' AND p.landuse <> 'none' AND ST_Isvalid(p.geom)
        ) as dtable
    order by area
) x
WHERE x.r = 1
GROUP BY bid, landuse;
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Property age

In [65]:
sql = """
INSERT INTO property_age (bid, age, area, city) 
SELECT x.bid, x.age, x.barea, '{city}'
FROM (
    SELECT bid, age, barea, ROW_NUMBER() OVER (PARTITION BY id ORDER BY area DESC) AS r
    from (
        SELECT p.pid, p.yearbuilt::int as age, b.bid, ST_Area(ST_Intersection(p.geom, b.geom)) as area, b.id, b.area as barea
        FROM temptable_{tempname} as p
        INNER JOIN building as b on ST_Intersects(p.geom, b.geom) AND NOT ST_Touches(p.geom, b.geom)
        WHERE b.city = '{city}' AND ST_Isvalid(p.geom) AND p.yearbuilt IS NOT NULL AND p.yearbuilt::int > 0
        ) as dtable
) x
WHERE x.r = 1 AND x.age <= 2014;
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Unused areas

In [67]:
unused_gdf = gpd.read_file('zip://../../data/boston/unused_areas/tl_2014_25_arealm.zip')
unused_gdf = unused_gdf[['geometry', 'MTFCC']]
unused_gdf = unused_gdf.to_crs({'init': 'epsg:4326'}) 

unused_gdf = unused_gdf[unused_gdf['MTFCC'].isin({'K2180', 'K2181', 'K2182', 'K2183', 'K2184', 'K2185',
                                                 'K2186', 'K2187', 'K2188', 'K2189', 'K2190'})].drop('MTFCC', axis=1)

unused_gdf['type'] = 'park'
unused_gdf['city'] = CITY
unused_gdf.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,geometry,type,city
25,"POLYGON ((-70.63465 41.60079, -70.63338 41.600...",park,boston1m
26,"POLYGON ((-69.99672 41.83456, -69.99654 41.834...",park,boston1m
27,"POLYGON ((-70.43617 41.62438, -70.43566 41.625...",park,boston1m
28,"POLYGON ((-70.51249 41.76387, -70.51205 41.763...",park,boston1m
29,"POLYGON ((-70.16208 41.68532, -70.16134 41.685...",park,boston1m


In [68]:
ins_gdf = process_geometry_SQL_insert(unused_gdf)
ins_gdf.to_sql('unused_areas', engine, if_exists='append', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

In [69]:
unused_gdf = gpd.read_file('zip://../../data/boston/unused_areas/tl_2014_25025_areawater.zip')
unused_gdf = unused_gdf[['geometry']]
unused_gdf = unused_gdf.to_crs({'init': 'epsg:4326'}) 
unused_gdf['type'] = 'water'
unused_gdf['city'] = CITY
unused_gdf.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,geometry,type,city
0,"POLYGON ((-71.07423 42.39064, -71.07420 42.390...",water,boston1m
1,"POLYGON ((-71.11066 42.35239, -71.11039 42.352...",water,boston1m
2,"POLYGON ((-71.07019 42.36619, -71.06933 42.367...",water,boston1m
3,"POLYGON ((-71.14366 42.26213, -71.14350 42.262...",water,boston1m
4,"POLYGON ((-71.14351 42.26152, -71.14344 42.261...",water,boston1m


In [70]:
ins_gdf = process_geometry_SQL_insert(unused_gdf)
ins_gdf.to_sql('unused_areas', engine, if_exists='append', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

In [71]:
unused_gdf = gpd.read_file('zip://../../data/boston/unused_areas/tl_2014_25025_areawater.zip')
unused_gdf = unused_gdf[['geometry']]
unused_gdf = unused_gdf.to_crs({'init': 'epsg:4326'}) 
unused_gdf['type'] = 'water'
unused_gdf['city'] = CITY
unused_gdf.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,geometry,type,city
0,"POLYGON ((-71.07423 42.39064, -71.07420 42.390...",water,boston1m
1,"POLYGON ((-71.11066 42.35239, -71.11039 42.352...",water,boston1m
2,"POLYGON ((-71.07019 42.36619, -71.06933 42.367...",water,boston1m
3,"POLYGON ((-71.14366 42.26213, -71.14350 42.262...",water,boston1m
4,"POLYGON ((-71.14351 42.26152, -71.14344 42.261...",water,boston1m


In [72]:
unused_gdf = gpd.read_file('../../data/boston/unused_areas/water_parks.geojson')
unused_gdf = unused_gdf[['geometry']]
unused_gdf = unused_gdf.to_crs({'init': 'epsg:4326'}) 
unused_gdf['type'] = 'parksrivers'
unused_gdf['city'] = CITY
unused_gdf.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,geometry,type,city
0,"POLYGON ((-71.16804 42.38389, -71.16888 42.384...",parksrivers,boston1m
1,"POLYGON ((-71.16186 42.39437, -71.16186 42.394...",parksrivers,boston1m
2,"POLYGON ((-71.11563 42.32323, -71.11637 42.323...",parksrivers,boston1m
3,"POLYGON ((-71.11938 42.29989, -71.11948 42.300...",parksrivers,boston1m
4,"POLYGON ((-71.09136 42.30924, -71.09153 42.309...",parksrivers,boston1m


In [73]:
ins_gdf = process_geometry_SQL_insert(unused_gdf)
ins_gdf.to_sql('temptable_{tempname}'.format(tempname=CITY.lower()), engine, if_exists='replace', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

In [74]:
sql = """
update temptable_{tempname} set geom=st_multi(st_buffer(geom, 0.0));
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [75]:
sql = """
DROP TABLE IF EXISTS temptable_unusedhelper_{tempname};
CREATE TEMPORARY TABLE temptable_unusedhelper_{tempname} AS
SELECT ST_Union(geom) as geom FROM unused_areas u 
WHERE city='{city}';

DROP TABLE IF EXISTS temptable_unusedhelper_exp_{tempname};
CREATE TEMPORARY TABLE temptable_unusedhelper_exp_{tempname} AS
SELECT (ST_Dump(geom)).geom FROM temptable_unusedhelper_{tempname} u;
CREATE INDEX ON temptable_unusedhelper_exp_{tempname} USING GIST (geom);
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [76]:
sql = """
update temptable_{tempname} t set geom=ST_Multi(st_buffer(ST_Difference(t.geom, h.geom), 0.0))
FROM temptable_unusedhelper_{tempname} h
WHERE st_intersects(t.geom, h.geom) AND (NOT ST_Touches(t.geom, h.geom)) AND ST_GeometryType(ST_Multi(ST_Difference(t.geom, h.geom))) <> 'ST_GeometryCollection';
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [77]:
sql = """
DELETE FROM temptable_{tempname} t 
USING temptable_unusedhelper_exp_{tempname} h
WHERE ST_Within(t.geom, h.geom) OR (st_intersects(t.geom, h.geom) AND (NOT ST_Touches(t.geom, h.geom)) AND ST_GeometryType(ST_Multi(ST_Difference(t.geom, h.geom))) = 'ST_GeometryCollection');
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [78]:
sql = """
update temptable_{tempname} set geom=st_multi(st_buffer(geom, 0.0));
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [79]:
sql = """
INSERT INTO unused_areas (geom, type, city) 
SELECT p.geom, p.type, p.city
FROM temptable_{tempname} as p
WHERE ST_Isvalid(p.geom) AND NOT EXISTS(SELECT * FROM unused_areas u WHERE ST_Intersects(u.geom, p.geom) AND u.city=p.city)
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Net area

In [80]:
unique_land_gdf = land_gdf.copy()
unique_land_gdf.loc[:, 'x'] = unique_land_gdf.geometry.centroid.x
unique_land_gdf.loc[:, 'y'] = unique_land_gdf.geometry.centroid.y
unique_land_gdf = unique_land_gdf.drop_duplicates(subset=['x', 'y'])

In [81]:
ins_gdf = process_geometry_SQL_insert(unique_land_gdf)
ins_gdf.to_sql('temptable_{tempname}'.format(tempname=CITY.lower()), engine, if_exists='replace', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

In [82]:
sql = """
UPDATE temptable_{tempname} p SET geom=ST_Multi(ST_buffer(p.geom, 0.0)) 
FROM
boundary b
WHERE b.city = '{city}' AND ST_Intersects(p.geom, b.geom) AND NOT ST_Touches(p.geom, b.geom);
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [83]:
## This deletes the blocks that are related to streets
sql = """
DELETE FROM block b
WHERE city='{city}' and NOT EXISTS (
    select 1 
    from temptable_{tempname} t 
    where st_intersects(t.geom, b.geom) and landuse <> 'none'  
    HAVING SUM(ST_Area(st_intersection(t.geom, b.geom)))/ST_Area(b.geom) > 0.2);
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [84]:
sql = """
DELETE 
FROM temptable_{tempname} t
USING unused_areas u 
WHERE u.city = '{city}' AND ST_Intersects(u.geom, t.geom) AND (NOT ST_Touches(u.geom, t.geom)) 
AND (ST_Contains(u.geom, t.geom) OR ST_AREA(ST_Intersection(t.geom, u.geom))/ST_Area(t.geom) > 0.5);
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [85]:
sql = """
INSERT INTO spatial_groups_net_area (sp_id, city, spatial_name, used_area) 
SELECT sp_id, city, spatial_name, SUM(ST_Area(ST_Intersection(s.approx_geom, t.geom)::geography))/1000000.
FROM temptable_{tempname} t
INNER JOIN spatial_groups s ON ST_Intersects(s.approx_geom, t.geom) AND NOT ST_Touches(s.approx_geom, t.geom)
WHERE s.city = '{city}' 
GROUP BY sp_id, city, spatial_name;
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## POIs

In [6]:
sql = """
REFRESH MATERIALIZED VIEW block_centroids;
"""

result = engine.execute(text(sql))

In [7]:
sql = """
REFRESH MATERIALIZED VIEW pois_requests;
"""

result = engine.execute(text(sql))

In [5]:
from collections import defaultdict
from joblib import Parallel, delayed

In [6]:
def make_trip(lon1, lat1, dest):
    # PERSONALIZE HERE
    r = requests.get(
        'http://localhost:5000/table/v1/foot/{lon1},{lat1};{dest}?annotations=distance&sources=0'.format(lon1=lon1, lat1=lat1, dest=dest))
    distances = r.json()['distances']
    distance = np.array(distances[0][1:], np.float32)
    distance[distance < 0] = 0
    return distance

def walkscore_list(bid, clon, clat, list_dests, ws, straight_distances):
    dists = make_trip(clon, clat, list_dests)
    straight_distances = np.array(straight_distances)
    #good_idxs = np.argwhere(dists < 1600)
    ws = np.array(ws)+ 0.00000001 #[good_idxs] 
    zeros_idxs = np.argwhere(dists == 0)
    dists[zeros_idxs] = 1
    if np.sum(ws) == 0 or len(ws) == 0:
        return bid, -1
    return bid, np.average(straight_distances/dists, weights=ws) #[good_idxs] [good_idxs]

cat_weights = {
    'grocery': [3],
    'Food': [.75,.45,.25,.25,.225,.225,.225,.225,.2,.2],
    'Shops': [.5,.45,.4,.35,.3],
    'Schools': [1],
    'Entertainment': [1],
    'Parks and outside': [1],
    'Coffee': [1.25,.75],
    'Banks': [1],
    'Books': [1]
}


def walkscore(meters):
	max_walk = 1500
	score = np.exp(-5 * (meters / max_walk) ** 2.5)
	score = np.clip(score, 0, 1)
	return score

def walkscore2_list(bid, clon, clat, list_dests, c):
    dists = make_trip(clon, clat, list_dests)
    #good_idxs = np.argwhere(dists < 1600)
    scores = np.sort(walkscore(dists))[::-1]
    n = len(cat_weights[c])
    d = np.zeros(n)
    d[:scores.shape[0]] = scores[:n]
    w = np.sum(np.array(d)*np.array(cat_weights[c]))
    assert w <= np.sum(cat_weights[c]) and w >= 0
    
    return bid, w #[good_idxs] [good_idxs]

In [7]:
sql = """
SELECT bid, lon, lat, dests, parent_cat FROM pois_requests WHERE city = '{city}'
""".format(city=CITY, tempname=CITY.lower())

blocks_df = pd.read_sql_query(sql, con=engine)
blocks_df.head()

,bid,lon,lat,dests,parent_cat
0,501173,-71.153999,42.339341,"-71.150413,42.336300;-71.149810,42.336201",Banks
1,501173,-71.155435,42.339306,"-71.150413,42.336300;-71.149810,42.336201",Banks
2,501173,-71.155896,42.339202,"-71.150413,42.336300;-71.149810,42.336201",Banks
3,501173,-71.152056,42.340222,"-71.150413,42.336300;-71.149810,42.336201",Banks
4,501173,-71.153048,42.339929,"-71.150413,42.336300;-71.149810,42.336201",Banks


In [8]:
sql = """
SELECT bid, COUNT(*) as size
FROM block_centroids b WHERE b.city = '{city}' 
GROUP BY bid
ORDER BY bid
""".format(city=CITY, tempname=CITY.lower())

n_blocks_df = pd.read_sql_query(sql, con=engine).set_index('bid')
n_blocks_df.head()

,size
bid,
501173,6
501174,7
501175,5
501176,3
501177,16


In [9]:
block_groups = defaultdict(list)
for index, row in blocks_df.iterrows():
    block_groups[row['bid']].append(row.values[1:])

In [10]:
from tqdm import tqdm

print(len(blocks_df))
results = [(idx, score) for idx, score in Parallel(n_jobs=10)(delayed(walkscore2_list)(bid, req[0], req[1], req[2], req[3]) for bid, reqs in tqdm(block_groups.items()) for req in reqs)]

  0%|          | 0/555 [00:00<?, ?it/s]

44246


100%|██████████| 555/555 [00:20<00:00, 27.60it/s]


In [11]:
block_vacuum_index = defaultdict(list)
bid2size = {k: v['size'] for k, v in n_blocks_df.iterrows()}

for bid, score in results:
    block_vacuum_index[bid].append(score)
    
sum_cat_weights = np.sum([y for x in cat_weights.values() for y in x])

for bid, score in block_vacuum_index.items():
    if len(score) > 0:
        score = (np.sum(score)/bid2size[bid])/sum_cat_weights
        assert score <= 1.01
        sql = "INSERT INTO walk_index (bid, score, city) VALUES ({}, {}, '{}')".format(bid, score, CITY)
        result = engine.execute(text(sql))

## Crime

In [12]:
df = pd.read_csv('../../data/boston/crime/crime-incident-reports-july-2012---august-2015-source-legacy-system.csv')
df.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,COMPNOS,NatureCode,INCIDENT_TYPE_DESCRIPTION,MAIN_CRIMECODE,REPTDISTRICT,REPORTINGAREA,FROMDATE,WEAPONTYPE,Shooting,DOMESTIC,SHIFT,Year,Month,DAY_WEEK,UCRPART,X,Y,STREETNAME,XSTREETNAME,Location
0,120420285.0,BERPTA,RESIDENTIAL BURGLARY,05RB,D4,629,07/08/2012 06:00:00 AM,Other,No,No,Last,2012,7,Sunday,Part One,763273.1791,2951498.962,ABERDEEN ST,NaN,"(42.34638135, -71.10379454)"
1,120419202.0,PSHOT,AGGRAVATED ASSAULT,04xx,B2,327,07/08/2012 06:03:00 AM,Firearm,Yes,No,Last,2012,7,Sunday,Part One,771223.1638,2940772.099,HOWARD AV,NaN,"(42.31684135, -71.07458456)"
2,120419213.0,ARMROB,ROBBERY,03xx,D4,625,07/08/2012 06:26:00 AM,Firearm,No,No,Last,2012,7,Sunday,Part One,765118.8605,2950217.536,JERSEY ST,QUEENSBERRY ST,"(42.34284135, -71.09698955)"
3,120419223.0,ALARMC,COMMERCIAL BURGLARY,05CB,B2,258,07/08/2012 06:56:00 AM,Other,No,No,Last,2012,7,Sunday,Part One,773591.8648,2940638.174,COLUMBIA RD,NaN,"(42.3164411, -71.06582908)"
4,120419236.0,ARMROB,ROBBERY,03xx,E18,496,07/08/2012 07:15:00 AM,Firearm,No,No,Last,2012,7,Sunday,Part One,759042.7315,2923832.681,COLLINS ST,NaN,"(42.27051636, -71.11989955)"


In [13]:
df = df[['FROMDATE', 'INCIDENT_TYPE_DESCRIPTION', 'Location', 'UCRPART']]
df.head()

,FROMDATE,INCIDENT_TYPE_DESCRIPTION,Location,UCRPART
0,07/08/2012 06:00:00 AM,RESIDENTIAL BURGLARY,"(42.34638135, -71.10379454)",Part One
1,07/08/2012 06:03:00 AM,AGGRAVATED ASSAULT,"(42.31684135, -71.07458456)",Part One
2,07/08/2012 06:26:00 AM,ROBBERY,"(42.34284135, -71.09698955)",Part One
3,07/08/2012 06:56:00 AM,COMMERCIAL BURGLARY,"(42.3164411, -71.06582908)",Part One
4,07/08/2012 07:15:00 AM,ROBBERY,"(42.27051636, -71.11989955)",Part One


In [14]:
print(df.count())
df = df.dropna()
print(df.count())
df.head()

FROMDATE                     268056
INCIDENT_TYPE_DESCRIPTION    268056
Location                     268056
UCRPART                      268056
dtype: int64
FROMDATE                     268056
INCIDENT_TYPE_DESCRIPTION    268056
Location                     268056
UCRPART                      268056
dtype: int64


,FROMDATE,INCIDENT_TYPE_DESCRIPTION,Location,UCRPART
0,07/08/2012 06:00:00 AM,RESIDENTIAL BURGLARY,"(42.34638135, -71.10379454)",Part One
1,07/08/2012 06:03:00 AM,AGGRAVATED ASSAULT,"(42.31684135, -71.07458456)",Part One
2,07/08/2012 06:26:00 AM,ROBBERY,"(42.34284135, -71.09698955)",Part One
3,07/08/2012 06:56:00 AM,COMMERCIAL BURGLARY,"(42.3164411, -71.06582908)",Part One
4,07/08/2012 07:15:00 AM,ROBBERY,"(42.27051636, -71.11989955)",Part One


In [15]:
df['datetime'] = pd.to_datetime(df['FROMDATE'].map(str).str.zfill(4), format='%m/%d/%Y  %I:%M:%S %p')
df.head()

,FROMDATE,INCIDENT_TYPE_DESCRIPTION,Location,UCRPART,datetime
0,07/08/2012 06:00:00 AM,RESIDENTIAL BURGLARY,"(42.34638135, -71.10379454)",Part One,2012-07-08 06:00:00
1,07/08/2012 06:03:00 AM,AGGRAVATED ASSAULT,"(42.31684135, -71.07458456)",Part One,2012-07-08 06:03:00
2,07/08/2012 06:26:00 AM,ROBBERY,"(42.34284135, -71.09698955)",Part One,2012-07-08 06:26:00
3,07/08/2012 06:56:00 AM,COMMERCIAL BURGLARY,"(42.3164411, -71.06582908)",Part One,2012-07-08 06:56:00
4,07/08/2012 07:15:00 AM,ROBBERY,"(42.27051636, -71.11989955)",Part One,2012-07-08 07:15:00


In [16]:
df['Location'] = df['Location'].str.replace('(', '')
df['Location'] = df['Location'].str.replace(')', '')

In [17]:
df['lng'] = df['Location'].str.split(', ').str[1].astype('float32')
df['lat'] = df['Location'].str.split(', ').str[0].astype('float32')
df['num'] = 1
df.head()

,FROMDATE,INCIDENT_TYPE_DESCRIPTION,Location,UCRPART,datetime,lng,lat,num
0,07/08/2012 06:00:00 AM,RESIDENTIAL BURGLARY,"42.34638135, -71.10379454",Part One,2012-07-08 06:00:00,-71.103798,42.346382,1
1,07/08/2012 06:03:00 AM,AGGRAVATED ASSAULT,"42.31684135, -71.07458456",Part One,2012-07-08 06:03:00,-71.074585,42.316841,1
2,07/08/2012 06:26:00 AM,ROBBERY,"42.34284135, -71.09698955",Part One,2012-07-08 06:26:00,-71.096992,42.342842,1
3,07/08/2012 06:56:00 AM,COMMERCIAL BURGLARY,"42.3164411, -71.06582908",Part One,2012-07-08 06:56:00,-71.065826,42.316441,1
4,07/08/2012 07:15:00 AM,ROBBERY,"42.27051636, -71.11989955",Part One,2012-07-08 07:15:00,-71.119896,42.270515,1


In [18]:
df['num'] = 1
df.head()

,FROMDATE,INCIDENT_TYPE_DESCRIPTION,Location,UCRPART,datetime,lng,lat,num
0,07/08/2012 06:00:00 AM,RESIDENTIAL BURGLARY,"42.34638135, -71.10379454",Part One,2012-07-08 06:00:00,-71.103798,42.346382,1
1,07/08/2012 06:03:00 AM,AGGRAVATED ASSAULT,"42.31684135, -71.07458456",Part One,2012-07-08 06:03:00,-71.074585,42.316841,1
2,07/08/2012 06:26:00 AM,ROBBERY,"42.34284135, -71.09698955",Part One,2012-07-08 06:26:00,-71.096992,42.342842,1
3,07/08/2012 06:56:00 AM,COMMERCIAL BURGLARY,"42.3164411, -71.06582908",Part One,2012-07-08 06:56:00,-71.065826,42.316441,1
4,07/08/2012 07:15:00 AM,ROBBERY,"42.27051636, -71.11989955",Part One,2012-07-08 07:15:00,-71.119896,42.270515,1


### Crime types

In [19]:
crime_types_df = pd.read_csv('../../data/crime_types/boston_types_categorized.csv')[['INCIDENT_TYPE_DESCRIPTION', 'UCRPART', 'UCR1']]
crime_types_df = crime_types_df[crime_types_df['UCRPART'] == 'Part One']
crime_types_df = crime_types_df.drop_duplicates()
crime_types_df.head()

,INCIDENT_TYPE_DESCRIPTION,UCRPART,UCR1
0,RESIDENTIAL BURGLARY,Part One,Burglary (breaking or entering)
1,AGGRAVATED ASSAULT,Part One,Aggravated assault
2,ROBBERY,Part One,Robbery
3,COMMERCIAL BURGLARY,Part One,Burglary (breaking or entering)
6,BENoProp,Part One,Larceny-theft (except motor vehicle theft)


In [20]:
crime_types_df.loc[crime_types_df['UCR1'] == 'Larceny-theft (except motor vehicle theft)', 'UCRPART'] = 'Part OneII'

In [21]:
crime_types_df

,INCIDENT_TYPE_DESCRIPTION,UCRPART,UCR1
0,RESIDENTIAL BURGLARY,Part One,Burglary (breaking or entering)
1,AGGRAVATED ASSAULT,Part One,Aggravated assault
2,ROBBERY,Part One,Robbery
3,COMMERCIAL BURGLARY,Part One,Burglary (breaking or entering)
6,BENoProp,Part OneII,Larceny-theft (except motor vehicle theft)
17,OTHER LARCENY,Part OneII,Larceny-theft (except motor vehicle theft)
18,AUTO THEFT,Part One,Motor vehicle theft
24,LARCENY FROM MOTOR VEHICLE,Part OneII,Larceny-theft (except motor vehicle theft)
52,HOMICIDE,Part One,Criminal homicide
62,ARSON,Part One,Arson


In [22]:
print(df['num'].count())
df = pd.merge(df.drop('UCRPART', axis=1), crime_types_df, on='INCIDENT_TYPE_DESCRIPTION')
print(df['num'].count())

268056
65294


In [23]:
df = df[df['UCRPART'] == 'Part One']

In [24]:
df[df['UCR1'] == 'Larceny-theft (except motor vehicle theft)']

,FROMDATE,INCIDENT_TYPE_DESCRIPTION,Location,datetime,lng,lat,num,UCRPART,UCR1


#### Subtypes of crimes

In [25]:
ucr_crimes_df = pd.read_csv('../../data/crime_types/UCR_crimes.csv')
ucr_crimes_df.head()

,Name,Category
0,Criminal homicide,Violent crime
1,Rape,Violent crime
2,Robbery,Violent crime
3,Aggravated assault,Violent crime
4,Burglary (breaking or entering),Property crime


In [26]:
df_ucr1 = pd.merge(df, ucr_crimes_df.rename(columns={'Name': 'UCR1'}), on='UCR1')

a = set(df_ucr1['UCR1'].drop_duplicates().values)
b = set(df['UCR1'].drop_duplicates().values)
assert(a.intersection(b) == a)

# Categories not present in crime dataset
df_ucr1[~(df_ucr1['UCR1'].isin(b))]

,FROMDATE,INCIDENT_TYPE_DESCRIPTION,Location,datetime,lng,lat,num,UCRPART,UCR1,Category


In [27]:
df = df_ucr1[df_ucr1['datetime'].dt.year == 2014]
df = df.rename(columns={'INCIDENT_TYPE_DESCRIPTION': 'description'})
df = df[['lng', 'lat', 'description', 'num', 'UCR1', 'Category']]
df.count()

lng            7751
lat            7751
description    7751
num            7751
UCR1           7751
Category       7751
dtype: int64

In [28]:
from geopandas import GeoDataFrame
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(df.lng, df.lat)]
df = df.drop(['lng', 'lng'], axis=1)
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
gdf.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,lat,description,num,UCR1,Category,geometry
3834,42.288853,RESIDENTIAL BURGLARY,1,Burglary (breaking or entering),Property crime,POINT (-71.07738 42.28885)
3835,42.345711,RESIDENTIAL BURGLARY,1,Burglary (breaking or entering),Property crime,POINT (-71.08559 42.34571)
3836,42.348404,RESIDENTIAL BURGLARY,1,Burglary (breaking or entering),Property crime,POINT (-71.13644 42.34840)
3837,42.325405,RESIDENTIAL BURGLARY,1,Burglary (breaking or entering),Property crime,POINT (-71.08803 42.32541)
3838,42.250256,RESIDENTIAL BURGLARY,1,Burglary (breaking or entering),Property crime,POINT (-71.13166 42.25026)


In [29]:
insert_gdf = process_geometry_SQL_insert(gdf)
insert_gdf.to_sql('temptable_{}'.format(CITY.lower()), engine, if_exists='replace', index=True, dtype={'geom': Geometry('Point', srid=4326)})

In [30]:
sql = """
insert into crime (sp_id, num, city, ucr1, ucr_category) 
select bid, SUM(num), '{city}', "UCR1", "Category" from(
SELECT num, bid, "UCR1", "Category", ROW_NUMBER() OVER (PARTITION BY index) AS r
from (
select c.index, c.num, b.bid, "UCR1", "Category"
from temptable_{tempname} as c
inner join blocks_group as b on ST_Intersects(b.geom, st_buffer(c.geom::geography, 30)::geometry)
where b.city='{city}'
    ) as dtable
) x
group by bid, "UCR1", "Category";
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Refresh materialized views

In [29]:
sql = """
REFRESH MATERIALIZED VIEW spatial_groups_unused_areas;
"""

result = engine.execute(text(sql))

In [30]:
sql = """
REFRESH MATERIALIZED VIEW block_building;
"""

result = engine.execute(text(sql))

In [31]:
sql = """
REFRESH MATERIALIZED VIEW blocks_group_with_building;
"""

result = engine.execute(text(sql))